In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd

In [2]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [3]:
print(sqlite3.version)

2.6.0


In [4]:
cur = con.cursor()

In [5]:
query1 = 'SELECT * FROM Facilities'
rs = cur.execute(query1)
facilities = pd.DataFrame(rs.fetchall())

facilities.columns = list(map(lambda x: x[0], rs.description))

In [6]:
facilities.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [7]:
query2 = 'SELECT * FROM Bookings'
rs = cur.execute(query2)
bookings = pd.DataFrame(rs.fetchall())

bookings.columns = list(map(lambda x: x[0], rs.description))

In [8]:
bookings.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [9]:
query1 = 'SELECT * FROM Members'
rs = cur.execute(query1)
members = pd.DataFrame(rs.fetchall())

members.columns = list(map(lambda x: x[0], rs.description))

In [10]:
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [15]:
def query(q):
    rs = cur.execute(q)
    df = pd.DataFrame(rs.fetchall())
    df.columns = list(map(lambda x: x[0], rs.description))
    print(df)

In [83]:
q = """WITH revenue AS(
SELECT F.name AS Facility_name, 
SUM(CASE WHEN B.memid=0 THEN B.slots*F.guestcost
     ELSE B.slots*F.membercost END) AS Revenue
FROM Bookings AS B
LEFT JOIN Facilities AS F
ON B.facid = F.facid
GROUP BY F.name
ORDER BY Revenue)

SELECT Facility_name, Revenue
FROM revenue
WHERE Revenue < 1000
"""

query(q)

   Facility_name  Revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


In [84]:
q = """SELECT (M1.surname || ',' || M1.firstname) AS Member, (M2.surname || ',' || M2.firstname) AS Recommender
FROM Members AS M1
Inner JOIN Members AS M2 on M1.memid = M2.recommendedby
ORDER BY Member, Recommender"""
query(q)

               Member              Recommender
0      Bader,Florence         Sarwin,Ramnaresh
1       Baker,Timothy              Coplin,Joan
2      Butters,Gerald          Genting,Matthew
3      Farrell,Jemima            Baker,Timothy
4      Farrell,Jemima             Pinker,David
5     Genting,Matthew         Rumney,Henrietta
6         Jones,David            Jones,Douglas
7     Joplette,Janice               Dare,Nancy
8     Joplette,Janice              Jones,David
9   Purview,Millicent                Hunt,John
10         Rownam,Tim               Boothe,Tim
11       Smith,Darren           Butters,Gerald
12       Smith,Darren          Joplette,Janice
13       Smith,Darren           Mackenzie,Anna
14       Smith,Darren             Owen,Charles
15       Smith,Darren               Smith,Jack
16        Smith,Tracy            Crumpet,Erica
17        Smith,Tracy        Purview,Millicent
18        Smith,Tracy  Worthington-Smyth,Henry
19    Stibbons,Ponder           Bader,Florence
20    Stibbon

In [95]:
q = """SELECT F.name AS Facility_name, 
SUM(CASE WHEN B.memid != 0 THEN B.slots
    ELSE 0 END) AS Total_usage
FROM Bookings AS B
LEFT JOIN Facilities AS F
ON B.facid = F.facid
GROUP BY F.name
ORDER BY Total_usage DESC"""
query(q)

     Facility_name  Total_usage
0  Badminton Court         1086
1   Tennis Court 1          957
2   Massage Room 1          884
3   Tennis Court 2          882
4    Snooker Table          860
5       Pool Table          856
6     Table Tennis          794
7     Squash Court          418
8   Massage Room 2           54


In [96]:
q = """SELECT F.name AS Facility_name, strftime('%m', DATE(B.starttime)) AS month, 
SUM(CASE WHEN B.memid != 0 THEN B.slots
    ELSE 0 END) AS Total_usage
FROM Bookings AS B
LEFT JOIN Facilities AS F
ON B.facid = F.facid
GROUP BY F.name, Month
ORDER BY Total_usage DESC"""""

query(q)

      Facility_name month  Total_usage
0   Badminton Court    09          507
1        Pool Table    09          443
2    Tennis Court 1    09          417
3   Badminton Court    08          414
4    Tennis Court 2    09          414
5     Snooker Table    09          404
6    Massage Room 1    09          402
7      Table Tennis    09          400
8    Tennis Court 2    08          345
9    Tennis Court 1    08          339
10   Massage Room 1    08          316
11    Snooker Table    08          316
12       Pool Table    08          303
13     Table Tennis    08          296
14   Tennis Court 1    07          201
15     Squash Court    08          184
16     Squash Court    09          184
17   Massage Room 1    07          166
18  Badminton Court    07          165
19    Snooker Table    07          140
20   Tennis Court 2    07          123
21       Pool Table    07          110
22     Table Tennis    07           98
23     Squash Court    07           50
24   Massage Room 2    09